In [55]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

In [57]:
data=pd.read_csv('historical_data.csv')

In [59]:
data=data.drop(columns={'Event Location','Round 1 Knockdown Total','Round 1 Significant Strike Total Attempted','Round 1 Significant Strike Total Landed',
                   'Round 1 Takedown Total Attempted','Round 1 Takedown Total Landed','Round 1 Submission Attempted','Round 1 Reversal',
                   'Round 1 Ground and Cage Control Time','Round 1 Significant Strike Head Attempted','Round 1 Significant Strike Head Landed',
                   'Round 1 Significant Strike Body Attempted','Round 1 Significant Strike Body Landed','Round 1 Significant Strike Leg Attempted',
                   'Round 1 Significant Strike Leg Landed','Round 1 Significant Strike Clinch Attempted',
                   'Round 1 Significant Strike Clinch Landed','Round 1 Significant Strike Ground Attempted',
                   'Round 1 Significant Strike Ground Landed','Round 2 Knockdown Total','Round 2 Significant Strike Total Attempted',
                   'Round 2 Significant Strike Total Landed','Round 2 Takedown Total Attempted','Round 2 Takedown Total Landed',
                   'Round 2 Submission Attempted','Round 2 Reversal','Round 2 Ground and Cage Control Time','Round 2 Significant Strike Head Attempted',
                   'Round 2 Significant Strike Head Landed','Round 2 Significant Strike Body Attempted','Round 2 Significant Strike Body Landed',
                   'Round 2 Significant Strike Leg Attempted','Round 2 Significant Strike Leg Landed','Round 2 Significant Strike Clinch Attempted',
                   'Round 2 Significant Strike Clinch Landed','Round 2 Significant Strike Ground Attempted','Round 2 Significant Strike Ground Landed',
                   'Round 3 Knockdown Total','Round 3 Significant Strike Total Attempted','Round 3 Significant Strike Total Landed',
                   'Round 3 Takedown Total Attempted','Round 3 Takedown Total Landed','Round 3 Submission Attempted','Round 3 Reversal',
                   'Round 3 Ground and Cage Control Time','Round 3 Significant Strike Head Attempted','Round 3 Significant Strike Head Landed',
                   'Round 3 Significant Strike Body Attempted','Round 3 Significant Strike Body Landed','Round 3 Significant Strike Leg Attempted',
                   'Round 3 Significant Strike Leg Landed','Round 3 Significant Strike Clinch Attempted','Round 3 Significant Strike Clinch Landed',
                   'Round 3 Significant Strike Ground Attempted','Round 3 Significant Strike Ground Landed','Round 4 Knockdown Total',
                   'Round 4 Significant Strike Total Attempted','Round 4 Significant Strike Total Landed','Round 4 Takedown Total Attempted',
                   'Round 4 Takedown Total Landed','Round 4 Submission Attempted','Round 4 Reversal','Round 4 Ground and Cage Control Time',
                   'Round 4 Significant Strike Head Attempted','Round 4 Significant Strike Head Landed','Round 4 Significant Strike Body Attempted',
                   'Round 4 Significant Strike Body Landed','Round 4 Significant Strike Leg Attempted','Round 4 Significant Strike Leg Landed',
                   'Round 4 Significant Strike Clinch Attempted','Round 4 Significant Strike Clinch Landed','Round 4 Significant Strike Ground Attempted',
                   'Round 4 Significant Strike Ground Landed','Round 5 Knockdown Total','Round 5 Significant Strike Total Attempted',
                   'Round 5 Significant Strike Total Landed','Round 5 Takedown Total Attempted','Round 5 Takedown Total Landed',
                   'Round 5 Submission Attempted','Round 5 Reversal','Round 5 Ground and Cage Control Time','Round 5 Significant Strike Head Attempted',
                   'Round 5 Significant Strike Head Landed','Round 5 Significant Strike Body Attempted','Round 5 Significant Strike Body Landed',
                   'Round 5 Significant Strike Leg Attempted','Round 5 Significant Strike Leg Landed','Round 5 Significant Strike Clinch Attempted',
                   'Round 5 Significant Strike Clinch Landed','Round 5 Significant Strike Ground Attempted','Round 5 Significant Strike Ground Landed'
                  })

In [61]:
# Convert Winning Time to seconds
def convert_time_to_seconds(time_str):
    # Check if the value is a valid string
    if isinstance(time_str, str):
        minutes, seconds = map(int, time_str.split(':'))
        return minutes * 60 + seconds
    else:
        # Return NaN if the value is not a valid string
        return None

# Apply the conversion safely
data['winning_time_seconds'] = data['Winning Time'].apply(convert_time_to_seconds)

# Handle missing values in the new column (optional)
data['winning_time_seconds'] = data['winning_time_seconds'].fillna(0)

# Apply the conversion
data['winning_time_seconds'] = data['Winning Time'].apply(convert_time_to_seconds)

# Feature: Is first-round win
data['is_first_round_win'] = (data['Winning Round'] == 1).astype(int)

# Feature: Normalized winning round
data['normalized_winning_round'] = data['Winning Round'] / data['Winning Round'].max()

# Feature: Time left in the round
round_duration = 300  # 5 minutes = 300 seconds
data['time_left_in_round'] = round_duration - data['winning_time_seconds']

# Feature: Fight duration percentage
total_possible_duration = data['Winning Round'].max() * round_duration
data['fight_duration_percentage'] = (
    ((data['Winning Round'] - 1) * round_duration + data['winning_time_seconds'])
    / total_possible_duration
)

# Display the new columns
print(data[['Winning Round', 'Winning Time', 'winning_time_seconds', 'is_first_round_win', 'normalized_winning_round', 'time_left_in_round', 'fight_duration_percentage']].head())


   Winning Round  Winning Time winning_time_seconds  is_first_round_win  \
0            1.0          0.22                 None                   1   
1            1.0          0.22                 None                   1   
2            1.0          1.37                 None                   1   
3            1.0          1.37                 None                   1   
4            1.0          4.03                 None                   1   

   normalized_winning_round time_left_in_round fight_duration_percentage  
0                       0.2                NaN                       NaN  
1                       0.2                NaN                       NaN  
2                       0.2                NaN                       NaN  
3                       0.2                NaN                       NaN  
4                       0.2                NaN                       NaN  


In [63]:
#converting height in cms
data['height'] = data['Height Feet'] * 30.48 + data['Height Inches']*2.54

In [65]:
#creating age column
data['Date of Birth'] = pd.to_datetime(data['Date of Birth'], format='mixed')
data['date'] = pd.to_datetime(data['date'], format='mixed')

dob = data['Date of Birth']
current_date = data['date']

data['age'] = (
    current_date.dt.year - dob.dt.year
    - ((current_date.dt.month < dob.dt.month) | ((current_date.dt.month == dob.dt.month) & (current_date.dt.day < dob.dt.day)))
)

print(data['age'])

0        25
1        28
2        23
3        26
4        25
         ..
12287    34
12288    38
12289    30
12290    28
12291    32
Name: age, Length: 12292, dtype: int32


In [67]:
data['Winner Last Name'] = data['Winner Last Name'].fillna('Unknown')
data['Fighter Last Name'] = data['Fighter Last Name'].fillna('Unknown')
mean_reach = data['Reach Inches'].dropna().mean()
data['Reach Inches'] = data['Reach Inches'].fillna(mean_reach)
data['Stance'] = data['Stance'].fillna('Unknown')

In [69]:
# Calculate total fight count for each fighter
fighter_fight_count = data.groupby('Fighter Last Name')['Fight ID'].nunique()
data['fighter_fight_count'] = data['Fighter Last Name'].map(fighter_fight_count)

# Calculate win count for each fighter
fighter_win_count = data[data['Winner Last Name'] == data['Fighter Last Name']].groupby('Fighter Last Name')['Fight ID'].nunique()
data['fighter_win_count'] = data['Fighter Last Name'].map(fighter_win_count)

# Fill NaN values in fighter_win_count with 0 (for fighters who haven't won yet)
data['fighter_win_count'] = data['fighter_win_count'].fillna(0)

# Display the resulting DataFrame with new columns
print(data[['Fighter Last Name', 'fighter_fight_count', 'fighter_win_count']].head())

# Calculate winning percentage
data['fighter_win_percentage'] = (data['fighter_win_count'] / data['fighter_fight_count']) * 100
data['fighter_win_percentage'] = data['fighter_win_percentage'].fillna(0)


  Fighter Last Name  fighter_fight_count  fighter_win_count
0          Sandoval                    5                2.0
1             Denis                    2                1.0
2         Schilling                    2                0.0
3            Pineda                   14                5.0
4            Hayden                    2                0.0


In [71]:
def calculate_implied_probability(odds):
    if odds < 0:  # Negative odds
        return abs(odds) / (abs(odds) + 100)
    else:  # Positive odds
        return 100 / (odds + 100)

# Apply the function to calculate probabilities
data['odds_probability'] = data['odds'].apply(calculate_implied_probability)

# Example Output
print(data[['odds', 'odds_probability']].head())

# Handle invalid or missing odds values
data['odds'] = pd.to_numeric(data['odds'], errors='coerce')  # Convert to numeric, setting invalid values to NaN
data['odds'] = data['odds'].fillna(data['odds'].median())  # Replace NaNs with median odds


     odds  odds_probability
0  236.00          0.297619
1 -292.86          0.745456
2  119.14          0.456329
3 -143.71          0.589676
4  262.86          0.275588


In [73]:
# Clean column names
data.columns = data.columns.str.strip().str.replace(' ', '_')

In [75]:
# Label encode categorical variables
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
data['Stance_encoded'] = encoder.fit_transform(data['Stance'])


In [77]:
# Now combining the rows for each fight
grouped = data.groupby('Fight_ID')

# Combine the rows for each fight
combined_rows = []
for fight_id, group in grouped:
    # Ensure each fight has exactly two rows
    if len(group) == 2:
        # Rename columns for the first fighter
        fighter_x = group.iloc[0].add_suffix('_x')
        # Rename columns for the second fighter
        fighter_y = group.iloc[1].add_suffix('_y')
        # Combine into a single row
        combined_row = pd.concat([fighter_x, fighter_y])
        combined_rows.append(combined_row)

# Create a new DataFrame from the combined rows
combined_df = pd.DataFrame(combined_rows)

# Reset the index
combined_df.reset_index(drop=True, inplace=True)

# Display the combined DataFrame
print(combined_df)

      Fight_ID_x                           Event_Title_x     date_x  \
0              1           UFC on FX: Guillard vs Miller 2012-01-20   
1              2           UFC on FX: Guillard vs Miller 2012-01-20   
2              3           UFC on FX: Guillard vs Miller 2012-01-20   
3              4           UFC on FX: Guillard vs Miller 2012-01-20   
4              5           UFC on FX: Guillard vs Miller 2012-01-20   
...          ...                                     ...        ...   
6141        6142  UFC Fight Night: Covington vs. Buckley 2024-12-14   
6142        6143  UFC Fight Night: Covington vs. Buckley 2024-12-14   
6143        6144  UFC Fight Night: Covington vs. Buckley 2024-12-14   
6144        6145  UFC Fight Night: Covington vs. Buckley 2024-12-14   
6145        6146  UFC Fight Night: Covington vs. Buckley 2024-12-14   

           Weight_Class_x Winning_Method_x  Winning_Round_x  Winning_Time_x  \
0            Bantamweight           KO/TKO              1.0         

In [79]:
combined_df['winner'] = (combined_df['Fighter_First_Name_x'] == combined_df['Winner_First_Name_x']).astype(int)

In [81]:
combined_df = combined_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [83]:
if 'date_x' in combined_df.columns:
    combined_df = combined_df.drop(columns=['date_x', 'date_y'], errors='ignore')

In [85]:
combined_df['weight_diff'] = combined_df['Weight_Pounds_x'] - combined_df['Weight_Pounds_y']
combined_df['height_diff'] = combined_df['height_x'] - combined_df['height_y']

In [89]:
combined_df.columns

Index(['Fight_ID_x', 'Event_Title_x', 'Weight_Class_x', 'Winning_Method_x',
       'Winning_Round_x', 'Winning_Time_x', 'Winner_First_Name_x',
       'Winner_Last_Name_x', 'Fighter_First_Name_x', 'Fighter_Last_Name_x',
       'Height_Feet_x', 'Height_Inches_x', 'Weight_Pounds_x', 'Reach_Inches_x',
       'Stance_x', 'Date_of_Birth_x', 'Knockdown_Total_x',
       'Significant_Strike_Total_Attempted_x',
       'Significant_Strike_Total_Landed_x', 'Takedown_Total_Attempted_x',
       'Takedown_Total_Landed_x', 'Submission_Attempted_x', 'Reversal_x',
       'Ground_and_Cage_Control_Time_x', 'Significant_Strike_Head_Attempted_x',
       'Significant_Strike_Head_Landed_x',
       'Significant_Strike_Body_Attempted_x',
       'Significant_Strike_Body_Landed_x',
       'Significant_Strike_Leg_Attempted_x', 'Significant_Strike_Leg_Landed_x',
       'Significant_Strike_Clinch_Attempted_x',
       'Significant_Strike_Clinch_Landed_x',
       'Significant_Strike_Ground_Attempted_x',
       'Signif

In [93]:
important_features = ['winning_time_seconds_x','Weight_Pounds_x',
 'Reach_Inches_x',
 'is_first_round_win_x',
 'normalized_winning_round_x',
 'time_left_in_round_x',
 'fight_duration_percentage_x',
 'height_x',
 'age_x',
 'fighter_fight_count_x',
 'fighter_win_count_x',
 'fighter_win_percentage_x',
 'odds_probability_x',
 'Stance_encoded_x',
                      
                     'winning_time_seconds_y','Weight_Pounds_y',
 'Reach_Inches_y',
 'is_first_round_win_y',
 'normalized_winning_round_y',
 'time_left_in_round_y',
 'fight_duration_percentage_y',
 'height_y',
 'age_y',
 'fighter_fight_count_y',
 'fighter_win_count_y',
 'fighter_win_percentage_y',
 'odds_probability_y',
 'Stance_encoded_y',
    'winner']

to_use_df = combined_df[important_features]


In [95]:
correlation = to_use_df[['height_x', 'Reach_Inches_x']].corr()
print("Corr_x : ", correlation)


Corr_x :                  height_x  Reach_Inches_x
height_x        1.000000        0.885742
Reach_Inches_x  0.885742        1.000000


In [97]:
for_reach = to_use_df.dropna(subset=['Reach_Inches_x', 'Reach_Inches_y'])

X_reach = for_reach[['height_x', 'height_y']]
y_reach = for_reach[['Reach_Inches_x', 'Reach_Inches_y']]

In [99]:
from sklearn.linear_model import LinearRegression
model_reach_X = LinearRegression()
model_reach_X.fit(X_reach[['height_x']], y_reach['Reach_Inches_x'])

model_reach_y = LinearRegression()
model_reach_y.fit(X_reach[['height_y']], y_reach['Reach_Inches_y'])

LinearRegression()

In [101]:
#Filling null values in reach taking height as a parameter in Linear Regressinon 

reach_x_pred = model_reach_X.predict(to_use_df[['height_x']])
reach_y_pred = model_reach_y.predict(to_use_df[['height_y']])

reach_x_pred = pd.Series(reach_x_pred, index=to_use_df.index)
reach_y_pred = pd.Series(reach_y_pred, index=to_use_df.index)

to_use_df['Reach_Inches_x'] = to_use_df['Reach_Inches_x'].fillna(reach_x_pred)
to_use_df['Reach_Inches_y'] = to_use_df['Reach_Inches_y'].fillna(reach_y_pred)


In [103]:
X = to_use_df.drop('winner', axis=1)
y = to_use_df['winner']

In [105]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(X)

In [107]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=72)

In [109]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(max_depth=7, random_state=72)
rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=7, random_state=72)

In [111]:
y_train_pred = rf.predict(X_train)

accuracy = accuracy_score(y_train, y_train_pred)
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(classification_report(y_train, y_train_pred))

Accuracy: 0.79
Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.78      0.79      2762
           1       0.79      0.80      0.80      2769

    accuracy                           0.79      5531
   macro avg       0.79      0.79      0.79      5531
weighted avg       0.79      0.79      0.79      5531



In [113]:
y_test_pred = rf.predict(X_test)

accuracy = accuracy_score(y_test, y_test_pred)
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(classification_report(y_test, y_test_pred))

Accuracy: 0.75
Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.72      0.75       317
           1       0.72      0.78      0.75       298

    accuracy                           0.75       615
   macro avg       0.75      0.75      0.75       615
weighted avg       0.75      0.75      0.75       615



In [ ]:
# Example: Check if any feature correlates with the target (winner)
correlation = to_use_df.corr()
print(correlation['winner'].sort_values(ascending=False))

# Remove features with a high correlation to 'winner'
# for instance, any column related to 'Winning Round' or 'Winning Method'


In [ ]:
data.columns

In [ ]:
# Create a column to check if the fighter is new
validation_data['is_new_fighter'] = ~validation_data['Fighter_Last_Name'].isin(train_data['Fighter_Last_Name'])

# For new fighters, use only physical features
physical_features = ['height', 'Weight_Pounds', 'Reach_Inches', 'age', 'Stance_encoded']
validation_data_new_fighter = validation_data[validation_data['is_new_fighter']][physical_features]

# For known fighters, use past data features and physical features
validation_data_known_fighter = validation_data[~validation_data['is_new_fighter']][important_features]


In [117]:
list(to_use_df.columns)

['winning_time_seconds_x',
 'Weight_Pounds_x',
 'Reach_Inches_x',
 'is_first_round_win_x',
 'normalized_winning_round_x',
 'time_left_in_round_x',
 'fight_duration_percentage_x',
 'height_x',
 'age_x',
 'fighter_fight_count_x',
 'fighter_win_count_x',
 'fighter_win_percentage_x',
 'odds_probability_x',
 'Stance_encoded_x',
 'winning_time_seconds_y',
 'Weight_Pounds_y',
 'Reach_Inches_y',
 'is_first_round_win_y',
 'normalized_winning_round_y',
 'time_left_in_round_y',
 'fight_duration_percentage_y',
 'height_y',
 'age_y',
 'fighter_fight_count_y',
 'fighter_win_count_y',
 'fighter_win_percentage_y',
 'odds_probability_y',
 'Stance_encoded_y',
 'winner']

In [119]:
# Columns to retain for validation (fighter names and physical features only)
validation_columns = [
    'height_x', 'height_y', 'age_x', 'age_y',          # Heights of both fighters
    'Reach_Inches_x', 'Reach_Inches_y', # Reach of both fighters
    'Weight_Pounds_x', 'Weight_Pounds_y', 'Stance_encoded_x', 'Stance_encoded_y' # Weight class and stances (if available)
]

# Validation dataset: only the columns relevant for future fights
validation_df = to_use_df[validation_columns]

# Training/Testing dataset: Keep all columns for modeling (excluding the target column 'winner')
train_test_df = to_use_df.copy()

# Check the shape of both datasets
print(f"Validation Dataset Shape: {validation_df.shape}")
print(f"Train/Test Dataset Shape: {train_test_df.shape}")


Validation Dataset Shape: (6146, 10)
Train/Test Dataset Shape: (6146, 29)


In [125]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Separate features and target from the train/test dataset
X = train_test_df.drop(columns=['winner'])
y = train_test_df['winner']

# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a RandomForest model
rf_model = RandomForestClassifier(max_depth=7, random_state=42)
rf_model.fit(X_train, y_train)

# Evaluate on the test dataset
y_test_pred = rf_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Test Accuracy: {test_accuracy:.2f}")
print("Classification Report on Test Data:")
print(classification_report(y_test, y_test_pred))

# Predict on the validation dataset (using only physical features)
validation_predictions = rf_model.predict(validation_df)

# Add predictions to the validation dataset for reference
validation_df['predicted_winner'] = validation_predictions

# Display sample predictions
print(validation_df.head())


Test Accuracy: 0.75
Classification Report on Test Data:
              precision    recall  f1-score   support

           0       0.74      0.74      0.74       595
           1       0.76      0.75      0.76       635

    accuracy                           0.75      1230
   macro avg       0.75      0.75      0.75      1230
weighted avg       0.75      0.75      0.75      1230



ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- predicted_winner


In [123]:
# Get the features used during training
training_features = X_train.columns

# Ensure validation dataset matches training features
for col in training_features:
    if col not in validation_df.columns:
        validation_df[col] = 0  # Fill missing columns with default values (e.g., 0)

# Reorder columns to match the training set
validation_df = validation_df[training_features]

# Predict outcomes for the validation dataset
validation_predictions = rf_model.predict(validation_df)

# Add predictions as a new column in the validation dataset
validation_df['predicted_winner'] = validation_predictions

# Display sample predictions
print(validation_df.head())


   winning_time_seconds_x  Weight_Pounds_x  Reach_Inches_x  \
0                       0              170            71.0   
1                       0              170            76.0   
2                       0              264            78.0   
3                       0              145            72.0   
4                       0              155            69.0   

   is_first_round_win_x  normalized_winning_round_x  time_left_in_round_x  \
0                     0                           0                     0   
1                     0                           0                     0   
2                     0                           0                     0   
3                     0                           0                     0   
4                     0                           0                     0   

   fight_duration_percentage_x  height_x  age_x  fighter_fight_count_x  ...  \
0                            0    180.34     28                      0  ...   
1     